<a href="https://colab.research.google.com/github/chiruconew/EXTRA/blob/main/MODELO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. CARGA DEL DATA SET Y CARGA DE LIBRERIAS INICIALES

In [340]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [341]:
url = 'https://raw.githubusercontent.com/chiruconew/EXTRA/main/data_set_dg.csv'
df  = pd.read_csv(url)

In [342]:
df.head()

,TELEFONO,CUOTA_BASICA,TIPO_PRODUCTO,CAPACIDAD_PLAN_DATOS,NIT,ANEXOS_HOME,ANEXOS_MOBILE,MESES_CLIENTE_ANTIGUEDAD,MESES_CONTRATO,DEPARTAMENTO_GUATEMALA,INICIO_DE_CONTRATO,SUBSIDIO,PAGO_DEVICE_CLIENTE,PROMEDIO_CONSUMO_DATOS_GB,CONSUMO_PORCENT_CUOTA,FACTURAS_PAGADAS_EN_TIEMPO,FACTURAS_PAGADAS_TARDE,CANTIDAD_ABONOS_PARA_PAGAR_AVG,AVG_DIAS_PAGO_A_TIEMPO,AVG_DIAS_PAGO_TARDE,DOWNSELL
0,52017983,198.9904,C,5,153912-4,0.0,1.0,246.0,18.0,GUATEMALA,06-OCT-16 00:00:00,NaN,NaN,1.0,0.18,0.0,10.0,1.0,0.0,-4.0,NO
1,55287011,499.0048,C,20,4930230-2,3.0,2.0,200.0,24.0,ESCUINTLA,07-JAN-19 00:00:00,3042.17,1607.14,10.0,0.51,8.0,2.0,1.0,2.0,-1.0,NO
2,53187495,298.9952,C,15,1199130-5,0.0,3.0,98.0,24.0,GUATEMALA,03-JAN-20 00:00:00,1237.57,207.36,5.0,0.33,10.0,0.0,1.0,8.0,0.0,NO
3,57813259,224.9968,Q,14,87103923,0.0,1.0,39.0,0.0,SOLOLA,21-AUG-20 00:00:00,NaN,NaN,18.0,1.29,4.0,4.0,1.0,17.0,-18.0,NO
4,40152016,298.9952,C,15,1526520-K,0.0,1.0,16.0,24.0,SAN MARCOS,15-NOV-19 00:00:00,1441.82,0.90,8.0,0.50,6.0,4.0,2.0,18.0,-5.0,NO


**BORRAMOS LOS NA**

In [343]:
df_recortado = df.dropna()

**CONVERTTIMOS LA CLASE EN NUMERICA**

In [344]:
df_recortado['DOWNSELL'].replace({"NO": 0, "SI": 1}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**ELIMINAMOS LAS COLUMNAS TELEFONO, NIT, e INICIO_DE_CONTRATO**

In [345]:
df_recortado = df_recortado.drop(['TELEFONO','NIT','INICIO_DE_CONTRATO'], axis=1)

In [346]:
df_recortado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10157 entries, 1 to 17466
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   CUOTA_BASICA                    10157 non-null  float64
 1   TIPO_PRODUCTO                   10157 non-null  object 
 2   CAPACIDAD_PLAN_DATOS            10157 non-null  int64  
 3   ANEXOS_HOME                     10157 non-null  float64
 4   ANEXOS_MOBILE                   10157 non-null  float64
 5   MESES_CLIENTE_ANTIGUEDAD        10157 non-null  float64
 6   MESES_CONTRATO                  10157 non-null  float64
 7   DEPARTAMENTO_GUATEMALA          10157 non-null  object 
 8   SUBSIDIO                        10157 non-null  float64
 9   PAGO_DEVICE_CLIENTE             10157 non-null  float64
 10  PROMEDIO_CONSUMO_DATOS_GB       10157 non-null  float64
 11  CONSUMO_PORCENT_CUOTA           10157 non-null  float64
 12  FACTURAS_PAGADAS_EN_TIEMPO      

##2.PREPROCESING

**Utilizamos ONE HOT ENCODE PARA EL TIPO DE PRODUCTO**

In [347]:
!pip install category_encoders

In [348]:
from category_encoders import OneHotEncoder

In [349]:
onehot_enc = OneHotEncoder( use_cat_names=True)
df_train_tipoprod = onehot_enc.fit_transform(df_recortado['TIPO_PRODUCTO'])
df_recortado = pd.concat([df_recortado, df_train_tipoprod], axis=1)


/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



In [350]:
df_recortado = df_recortado.drop(['TIPO_PRODUCTO'], axis=1)

**Utlilizamos en Frequency para el conteo de Departamentos**

In [351]:
from category_encoders import CountEncoder
count_enc_depto = CountEncoder( combine_min_nan_groups=True, min_group_size=35,min_group_name='OTROS')
df_recortado['DEPARTAMENTO_TRANS'] = count_enc_depto.fit_transform(df_recortado['DEPARTAMENTO_GUATEMALA'])


In [352]:
df_recortado = df_recortado.drop(['DEPARTAMENTO_GUATEMALA'], axis=1)
df_recortado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10157 entries, 1 to 17466
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   CUOTA_BASICA                    10157 non-null  float64
 1   CAPACIDAD_PLAN_DATOS            10157 non-null  int64  
 2   ANEXOS_HOME                     10157 non-null  float64
 3   ANEXOS_MOBILE                   10157 non-null  float64
 4   MESES_CLIENTE_ANTIGUEDAD        10157 non-null  float64
 5   MESES_CONTRATO                  10157 non-null  float64
 6   SUBSIDIO                        10157 non-null  float64
 7   PAGO_DEVICE_CLIENTE             10157 non-null  float64
 8   PROMEDIO_CONSUMO_DATOS_GB       10157 non-null  float64
 9   CONSUMO_PORCENT_CUOTA           10157 non-null  float64
 10  FACTURAS_PAGADAS_EN_TIEMPO      10157 non-null  float64
 11  FACTURAS_PAGADAS_TARDE          10157 non-null  float64
 12  CANTIDAD_ABONOS_PARA_PAGAR_AVG  

**VAMOS A SEPARAR EN TRAIN Y TEST**

Necesitamos convertir el dataframe en ARRAY DE NUMPY para poder utilizar la funcion de Stratify y partir un dataset Desbalanceado

In [353]:
import numpy as np

In [354]:
from sklearn.model_selection import train_test_split

In [355]:
X_df =  df_recortado.drop(['DOWNSELL'], axis=1)

In [356]:
y_df = df_recortado['DOWNSELL']

In [357]:
X = X_df.to_numpy()
y = y_df.to_numpy()

In [358]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.2,stratify = y)

**Usaremos una regresion Logistica, basada en el X_train y X_test, que separamos anteriormente**

In [359]:
from sklearn.linear_model import LogisticRegression
regresion_logistica = LogisticRegression( max_iter=20000)
regresion_logistica.fit(X,y)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=20000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [360]:
score = regresion_logistica.score(X, y)
print("Accuracy: ",f"{score}")

Accuracy:  0.9454563355321454


**Con la regrersion logistica no Converge, asumimos que es a raiz de que las variables numericas, aun no han sido normalizadas**

In [361]:
from sklearn.tree import DecisionTreeClassifier

In [362]:
tree = DecisionTreeClassifier()
tree.fit(X_df, y_df)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

## TRANSFORMATION

**NOrmalizamos las variables numericas, con el proposito que converga**

Iniciamos con la Capacidad del Plan de Datos, con ONE HOT ENCONDER

In [363]:
from category_encoders import OneHotEncoder

In [364]:
X_df['CAP_PLAN_ENC'] = df_recortado['CAPACIDAD_PLAN_DATOS'].astype(str)
onehot_enc = OneHotEncoder( use_cat_names=True)
X_caplan = onehot_enc.fit_transform(X_df['CAP_PLAN_ENC'])
X_df = pd.concat([X_df, X_caplan], axis=1)
X_df = X_df.drop(['CAPACIDAD_PLAN_DATOS','CAP_PLAN_ENC'], axis=1)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



Vamos a transformar la Cuota Basica

In [365]:
from sklearn.preprocessing import MinMaxScaler

In [366]:
scaler = MinMaxScaler()
col_name = 'CUOTA_BASICA'
trans_col_name = 'CUOTA_BASICA_MM'
X_df[trans_col_name] = scaler.fit_transform(X_df[[col_name]])
X_df = X_df.drop(['CUOTA_BASICA'], axis=1)

Pasaremos las variables Anexos Home y Anexos Movil que son numericas, las convertiremos en variables Categoricas

In [367]:
X_df.loc[X_df['ANEXOS_MOBILE'] == 0, 'ANEXOS_TRANS'] = 0
X_df.loc[X_df['ANEXOS_MOBILE'] != 0, 'ANEXOS_TRANS'] = 1
X_df.loc[X_df['ANEXOS_HOME'] == 0, 'ANEXOS_TRANS_2'] = 0
X_df.loc[X_df['ANEXOS_HOME'] != 0, 'ANEXOS_TRANS_2'] = 1
X_df['ANEXOS_TRANS_V'] = X_df['ANEXOS_TRANS'] + X_df['ANEXOS_TRANS_2']

In [368]:
X_df = X_df.drop(['ANEXOS_MOBILE','ANEXOS_HOME'], axis=1)

Transformaremos los meses de antiguedad del cliente a una distribucion lo mas normal posible a traves de Power Transform Yeo-Johnson

In [369]:
from sklearn.preprocessing import PowerTransformer

In [370]:
scaler = PowerTransformer()
col_name ='MESES_CLIENTE_ANTIGUEDAD'
trans_col_name='MESES_CLIENTE_ANTIGUEDAD_PW'
X_df[trans_col_name] = scaler.fit_transform(X_df[[col_name]])
X_df = X_df.drop(['MESES_CLIENTE_ANTIGUEDAD'], axis=1)

Vamos a convertir los meses de contrato del cliente en una variable categorica a traves de CountEncoder

In [371]:
from category_encoders import CountEncoder

In [372]:
X_df.loc[X_df['MESES_CONTRATO'] == 12, 'MESES_CONT_NAME'] = 'DOCE'
X_df.loc[X_df['MESES_CONTRATO'] == 24, 'MESES_CONT_NAME'] = 'VEINTICUATRO'
X_df.loc[X_df['MESES_CONTRATO'] == 18, 'MESES_CONT_NAME'] = 'DIECIOCHO'

In [373]:
count_enc_meses = CountEncoder(normalize=True)

In [374]:
X_df['MESES_CONT_TRANS'] = count_enc_meses.fit_transform(X_df['MESES_CONT_NAME'])
X_df = X_df.drop(['MESES_CONT_NAME','MESES_CONTRATO'], axis=1)

Vamos transfomar el Subisidio a traves de QuantileTransformer

In [375]:
from sklearn.preprocessing import QuantileTransformer
subsidio_scaler = QuantileTransformer( output_distribution='normal')
col_name = 'SUBSIDIO'
trans_col_name = 'SUBSIDIO_QT'
X_df[trans_col_name] = subsidio_scaler.fit_transform(X_df[[col_name]])
X_df = X_df.drop(['SUBSIDIO'], axis=1)

Vamos transfomar la variable PAGO DEVICE CLIENTE, a traves de POWER TRANSFORM

In [376]:
X_df['PAGO_DEVICE_CLIENTE'] = X_df['PAGO_DEVICE_CLIENTE'].abs() 
pagoCliente_scaler = PowerTransformer()
col_name = 'PAGO_DEVICE_CLIENTE'
trans_col_name = 'PAGO_DEVICE_CLIENTE_PT'
X_df[trans_col_name] = pagoCliente_scaler.fit_transform(X_df[[col_name]])
X_df = X_df.drop(['PAGO_DEVICE_CLIENTE'], axis=1)

Vamos a transformar la variable AVG_DIAS_PAGO_A_TIEMPO y AVG_DIAS_PAGO_TARDE con MinMax Scale

In [377]:
scaler = MinMaxScaler()

col_name = 'AVG_DIAS_PAGO_TARDE'
trans_col_name = 'AVG_DIAS_PAGO_TARDE_MM'
X_df[trans_col_name] = scaler.fit_transform(X_df[[col_name]])
X_df = X_df.drop(['AVG_DIAS_PAGO_TARDE'], axis=1)

col_name = 'AVG_DIAS_PAGO_A_TIEMPO'
trans_col_name = 'AVG_DIAS_PAGO_A_TIEMPO_MM'
X_df[trans_col_name] = scaler.fit_transform(X_df[[col_name]])
X_df = X_df.drop(['AVG_DIAS_PAGO_A_TIEMPO'], axis=1)



Vamos a crear una nueva variable categorica a partir de las variables numericas FACTURAS_PAGADAS_EN_TIEMPO y FACTURAS_PAGADAS_TARDE, y quedara con un RATIO

In [378]:
X_df['POR_PAGO_FACTURAS'] = X_df['FACTURAS_PAGADAS_EN_TIEMPO'] / (X_df['FACTURAS_PAGADAS_TARDE'] + X_df['FACTURAS_PAGADAS_EN_TIEMPO'])
X_df = X_df.drop(['FACTURAS_PAGADAS_EN_TIEMPO','FACTURAS_PAGADAS_TARDE'], axis=1)


In [379]:
X_df = X_df.drop(['CANTIDAD_ABONOS_PARA_PAGAR_AVG','PROMEDIO_CONSUMO_DATOS_GB'], axis=1)

**Probaremos con el modelo de Regresion LIneal Logistica con variables transformadas para ver si converge**

In [380]:
X = X_df.to_numpy()
y = y_df.to_numpy()

In [381]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.2,stratify = y)

In [382]:
regresion_logistica = LogisticRegression( max_iter=20000)
regresion_logistica.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=20000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [383]:
score = regresion_logistica.score(X, y)
print("Accuracy: ",f"{score}")

Accuracy:  0.9501821403957862
